In [1]:
import numpy as np

def DiscreteFourierTransform(x):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    n = np.arange(N)
    k = n.reshape((N, 1))
    M = np.exp(-2j * np.pi * k * n / N)
    return np.dot(M, x)

In [2]:
x = np.random.random(1024)
np.allclose(DiscreteFourierTransform(x), np.fft.fft(x))

True

In [4]:
%timeit DiscreteFourierTransform(x)
%timeit np.fft.fft(x)

98.4 ms ± 3.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
20.8 µs ± 449 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [15]:
def fft(x):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    if N % 2 > 0:
        raise ValueError("must be a power of 2")
    elif N <= 2:
        return DiscreteFourierTransform(x)
    else:
        X_even = fft(x[::2])
        X_odd = fft(x[1::2])
        terms = np.exp(-2j * np.pi * np.arange(N) / N)
        return np.concatenate([X_even + terms[:int(N/2)] * X_odd,
                               X_even + terms[int(N/2):] * X_odd])

In [16]:
x = np.random.random(1024)
np.allclose(fft(x), np.fft.fft(x))

True

In [18]:
%timeit DiscreteFourierTransform(x)
%timeit fft(x)
%timeit np.fft.fft(x)

99.8 ms ± 3.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
14.5 ms ± 313 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
20.2 µs ± 815 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [19]:
def fft_vector(x):
        x = np.asarray(x, dtype=float)
        N = x.shape[0]
        if np.log2(N) % 1 > 0:
            raise ValueError("must be a power of 2")
        
        N_min = min(N, 2)
    
        n = np.arange(N_min)
        k = n[:, None]
        M = np.exp(-2j * np.pi * n * k / N_min)
        X = np.dot(M, x.reshape((N_min, -1)))
        while X.shape[0] < N:
            X_even = X[:, :int(X.shape[1] / 2)]
            X_odd = X[:, int(X.shape[1] / 2):]
            terms = np.exp(-1j * np.pi * np.arange(X.shape[0])
                        / X.shape[0])[:, None]
        X = np.vstack([X_even + terms * X_odd, X_even - terms * X_odd])
        return X.ravel()

In [ ]:
%timeit fft(x)
%timeit fft_vector(x)
%timeit np.fft.fft(x)

15.4 ms ± 1.04 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
